https://www.1point3acres.com/bbs/thread-1086943-1-1.html

### Problem
题干如下：
subscription: list of users {name, plan, begin_date, duration}，要求按顺序发email (plan当天发welcome, -15 days发upcoming expiration，expire当天发expire)

例如：
```python
users = [{name: A, plan: X, begin_date = 0, duration = 30}, {name: B, plan: Y, begin_date = 1, duration = 15}]
```
输出
```
0: [Welcome] A, subscribe in plan X
1: [Welcome] B, subscribe in plan Y
1: [Upcoming expiration] B, subscribe in plan Y
15: [Upcoming expiration] A, subscribe in plan X
16: [Expired] B, subcribe in plan Y
30: [Expired] A, subscribe in plan X
```

第二问：基础上增加一个change in plans, list of changes {name, new_plan, change_date}
第三问是bonus，第二问的基础上再加一个renew，change list中entry为 {name, extension, change_date}


In [9]:
users = [
    {'name': 'A', 'plan': 'X', 'begin_date': 0, 'duration': 30}, 
    {'name': 'B', 'plan': 'Y', 'begin_date': 1, 'duration': 15}
]

In [23]:
def email_subscription(users):
    email_list = []

    for user in users:
        # add welcome email
        welcome_email = f"{user['begin_date']}: [Welcome] {user['name']}, subscribe in plan {user['plan']}"
        print(welcome_email)

        # add upcoming expiration email
        reminder_email = f"{user['begin_date'] + user['duration'] - 15 + 1}: [Upcoming expiration] {user['name']}, subscribe in plan {user['plan']}"
        print(reminder_email)

        # add expiration email
        exp_email = f"{user['begin_date'] + user['duration']}: [Expired] {user['name']}, subscribe in plan {user['plan']}"
        print(exp_email)





In [24]:
email_subscription(users)

0: [Welcome] A, subscribe in plan X
16: [Upcoming expiration] A, subscribe in plan X
30: [Expired] A, subscribe in plan X
1: [Welcome] B, subscribe in plan Y
2: [Upcoming expiration] B, subscribe in plan Y
16: [Expired] B, subscribe in plan Y
